#### Deep Learning

+ A model processes data and lables 
+ The model optimizes through a training loop
+ New dummy data is generated for each training loop
+ Able to use multiple models (standard is FNN)
+ Introduce Transformer model

TODO:
+ Show the confidence of a prediction (with softmax probability between 0 and 1) 
+ Compare multiple models on dummy data (ML, FNN, CNN, RNN, Transformer(Encoder))

#### Imports

In [1]:
import sklearn.model_selection as sk
import numpy as np   
import matplotlib.pyplot as plt
import os              
import sys
import random
import tensorflow as tf
import keras
from keras import layers
from IPython.display import clear_output

main_path = os.path.dirname(os.getcwd())

eegyolk_path = os.path.join(main_path, 'eegyolk')
sys.path.insert(0, eegyolk_path)
from eegyolk import dummy_data_functions as dummy
from eegyolk import display_helper as disp

from models import transfomer

#### Initialise Model

Feedforward neural network (FNN) 

In [2]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1024,)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])

model.compile(optimizer='sgd', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))

#### Transformer model:

Run *training loop* again after this cell, to use this transformer model instead of the feedforward NN.

The model is originally from:
+ Author: Bruce Shuyue Jia
+ Source: https://github.com/SuperBruceJia/EEG-DL/blob/master/Models/main-Transformer.py

In [5]:
model = transfomer.TransformerModel()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="binary_crossentropy")
              # metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall()]
              

#### Function for creating batch of data with mixed labels

In [3]:
batch_size = 128

planck_distribution = dummy.generate_frequency_distribution("planck")
const_distribution = dummy.generate_frequency_distribution("constant")

def create_batch(batch_size):
  X = []
  Y = np.zeros(batch_size)

  for i in range(batch_size):
      if random.random() < 0.5:
          X.append(dummy.generate_epoch(const_distribution))
      else:
          X.append(dummy.generate_epoch(planck_distribution))
          Y[i] = 1

  return X, Y

#### Training loop:

In [8]:
for j in range(4):
  X_train, Y_train = create_batch(batch_size)
  model.fit(np.array(X_train), Y_train, epochs=4, batch_size=10)

  X_test, Y_test = create_batch(batch_size)
  loss = model.evaluate(np.array(X_test),  Y_test, verbose=2)

  clear_output(wait=True)


Epoch 1/4
13/13 [==============================] - 0s 8ms/step - loss: 0.0640 - precision: 1.0000 - binary_accuracy: 0.9922 - recall: 0.9853
Epoch 2/4
13/13 [==============================] - 0s 8ms/step - loss: 0.0747 - precision: 0.9710 - binary_accuracy: 0.9766 - recall: 0.9853
Epoch 3/4
13/13 [==============================] - 0s 8ms/step - loss: 0.1054 - precision: 0.9848 - binary_accuracy: 0.9688 - recall: 0.9559
Epoch 4/4
13/13 [==============================] - 0s 8ms/step - loss: 0.0651 - precision: 0.9855 - binary_accuracy: 0.9922 - recall: 1.0000
4/4 - 0s - loss: 0.0064 - precision: 1.0000 - binary_accuracy: 1.0000 - recall: 1.0000 - 46ms/epoch - 11ms/step


#### Plot Loss

In [11]:
loss?

Type:        list
String form: [0.006358501501381397, 1.0, 1.0, 1.0]
Length:      4
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.
